In this notebook, we will calculate features using Trotterization with a noisy classical simulator (AerSimulator)

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
# This is necessary only when running the notebook in the docker container
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']
/home/jovyan/fourier_learning_ibm


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
import warnings

warnings.filterwarnings("ignore")

### Config backend 

In [5]:
backend_qpu, backend_sim_noiseless, backend_sim_noisy = setup_backend(
    qpu_name="ibm_marrakesh"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend noiseless simulator: AerSimulator('aer_simulator')

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['cz', 'id', 'reset', 'x', 'sx', 'measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155

### Prepare parameters and load dataset

In [6]:
n_qubits = 4
# C = 3 * (n_qubits - 1)
# C = 3
C = 5
threshold = 0.6

# Load dataset (orient="records")
df_n_step = pd.read_json(
    f"results/get_param/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
    orient="records",
)
n_samples = 55

# Note that K = n_features-1.
# n_features counts only the number of real features, also the one of t=0, i.e., constant term.
# The number of all features, including imaginary numbers, is 2K+1.
n_features = len(df_n_step)

times = df_n_step["t"].values

display(df_n_step)
print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_features: {n_features}")
print(f"times: {times}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json", orient="records"
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

,t,n_step,sta_fidelity
0,0.000000,1,1.000000
1,0.628319,1,0.960905
2,1.256637,1,0.934116
3,1.884956,2,0.923847
4,2.513274,2,0.899309
5,3.141593,3,0.878686
6,3.769911,3,0.862589
7,4.398230,4,0.840110
8,5.026548,5,0.813450
9,5.654867,5,0.800024


n_qubits: 4
C: 5
threshold: 0.6
n_samples: 55
n_features: 19
times: [ 0.          0.62831853  1.25663706  1.88495559  2.51327412  3.14159265
  3.76991118  4.39822972  5.02654825  5.65486678  6.28318531  6.91150384
  7.53982237  8.1681409   8.79645943  9.42477796 10.05309649 10.68141502
 11.30973355]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.3949339961, -0.0881126084, 0.5169533955]",5.547457,5.547450,6.426300e-06
1,1,"[0.1829492443, -0.3761751478, 0.4408756079]",3.841905,3.841881,2.462770e-05
2,2,"[0.2841284266, 0.3112471996, -0.4046243738]",1.129636,1.129639,3.412100e-06
3,3,"[-0.0819280326, -0.2133520868, 0.7047198806]",4.731457,4.731416,4.172410e-05
4,4,"[0.2749649723, 0.616884329, -0.1081506987]",0.852425,0.852410,1.459610e-05
5,5,"[-0.3572377034, 0.0714901473, -0.5712721494]",0.828226,0.828227,1.070800e-06
6,6,"[0.4567030859, 0.1835342376, 0.3597626765]",3.697788,3.697784,4.380700e-06
7,7,"[-0.1441345638, 0.4663640207, 0.3895014155]",1.258985,1.258995,1.064270e-05
8,8,"[0.45117141930000004, -0.49489387500000004, -0...",3.279570,3.279566,4.387400e-06
9,9,"[-0.4631642178, -0.3509909223, 0.1858448599000...",1.577586,1.577586,7.130000e-08


# Calculate Fourier feature

## Trotter simulation (Simulator, Noise free)

In [7]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": df_n_step["n_step"].values,
    "backend": backend_sim_noisy,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [8]:
# For noisy simulator, we need job instances instead of job ids
_, jobs, _ = run_job(config)

Preparing circuits for sample 0/55
Preparing circuits for sample 1/55
Preparing circuits for sample 2/55
Preparing circuits for sample 3/55
Preparing circuits for sample 4/55
Preparing circuits for sample 5/55
Preparing circuits for sample 6/55
Preparing circuits for sample 7/55
Preparing circuits for sample 8/55
Preparing circuits for sample 9/55
Preparing circuits for sample 10/55
Preparing circuits for sample 11/55
Preparing circuits for sample 12/55
Preparing circuits for sample 13/55
Preparing circuits for sample 14/55
Preparing circuits for sample 15/55
Preparing circuits for sample 16/55
Preparing circuits for sample 17/55
Preparing circuits for sample 18/55
Preparing circuits for sample 19/55
Preparing circuits for sample 20/55
Preparing circuits for sample 21/55
Preparing circuits for sample 22/55
Preparing circuits for sample 23/55
Preparing circuits for sample 24/55
Preparing circuits for sample 25/55
Preparing circuits for sample 26/55
Preparing circuits for sample 27/55
Pr

In [9]:
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_14 Im,f_15 Re,f_15 Im,f_16 Re,f_16 Im,f_17 Re,f_17 Im,f_18 Re,f_18 Im,expected_value
0,0,0.976562,0.536754,0.366397,0.068647,0.134995,-0.104079,-0.078060,-0.302837,0.134123,...,-0.460231,0.224108,-0.190572,0.075469,-0.020433,0.164027,-0.080277,0.287337,0.185405,5.547457
1,1,0.975586,0.623469,0.433023,0.109347,0.349517,-0.237028,0.037435,-0.219611,-0.183774,...,-0.362689,0.182847,-0.334394,0.175629,0.041914,-0.073325,0.264588,-0.448067,0.205847,3.841905
2,2,0.955078,0.741202,-0.191559,0.283247,-0.173779,-0.026473,-0.234176,-0.149654,-0.266701,...,-0.009276,0.033685,0.076411,-0.098358,0.081242,-0.091913,0.078019,0.097950,-0.119032,1.129636
3,3,0.961914,0.513096,0.253781,-0.074823,-0.138312,-0.291161,-0.628267,-0.034131,-0.649698,...,0.141384,-0.020771,0.243600,0.092774,0.212848,0.177771,0.133096,0.141052,0.072304,4.731457
4,4,0.958984,0.826694,-0.256224,0.569060,-0.398222,0.350405,-0.525225,0.174270,-0.446475,...,-0.210183,-0.077518,-0.158420,-0.030754,-0.058831,-0.011602,-0.012501,-0.050403,-0.040924,0.852425
5,5,0.958984,0.498016,-0.380474,0.019324,-0.098019,-0.041136,0.133062,-0.209192,-0.074819,...,0.455821,0.175954,-0.003115,-0.126092,-0.074375,-0.093484,0.050255,0.038259,-0.036082,0.828226
6,6,0.962891,0.628900,0.262508,0.145245,0.175545,0.181153,-0.023948,0.134280,0.185678,...,-0.241122,-0.098633,-0.393555,0.251731,-0.163580,0.358650,0.217161,0.258730,0.252730,3.697788
7,7,0.969727,0.793235,-0.107501,0.473637,-0.140743,0.226622,-0.305122,0.067466,-0.296008,...,0.144858,0.171269,0.080097,0.304074,-0.026373,0.351619,-0.089067,0.297469,-0.121627,1.258985
8,8,0.960938,0.632429,0.455654,0.169462,0.373160,-0.233013,0.179125,-0.247012,-0.050800,...,0.015468,0.189406,0.247332,-0.060112,0.277413,-0.114360,0.154814,0.040401,-0.080467,3.279570
9,9,0.959961,0.727915,0.043035,0.378237,0.043006,0.030648,0.243728,-0.077971,0.333347,...,-0.344625,0.248375,-0.165954,0.469335,-0.009780,0.537972,0.037220,0.428038,0.010504,1.577586
